# Исследование надежности заемщиков.

**Цель исследования: разобраться, влияет ли уровень дохода, семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.**

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
debts = pd.read_csv('/datasets/data.csv')
display(debts.sample(10))
debts.info()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3718,0,-492.613197,32,НЕОКОНЧЕННОЕ ВЫСШЕЕ,2,гражданский брак,1,F,компаньон,0,129250.032264,сделка с автомобилем
3784,0,-1070.790684,48,высшее,0,женат / замужем,0,M,компаньон,0,193971.031455,операции с коммерческой недвижимостью
21456,2,-2247.862982,23,среднее,1,женат / замужем,0,F,сотрудник,1,115291.719953,получение дополнительного образования
12002,2,-102.287139,23,высшее,0,гражданский брак,1,F,сотрудник,0,97699.528037,получение дополнительного образования
19503,1,398715.819476,54,Высшее,0,женат / замужем,0,M,пенсионер,0,553313.883070,строительство жилой недвижимости
8018,0,-660.488766,41,высшее,0,гражданский брак,1,F,компаньон,1,118169.175009,на проведение свадьбы
15695,0,-3806.323142,30,среднее,1,Не женат / не замужем,4,M,сотрудник,0,169824.896961,сделка с автомобилем
21043,0,-1755.222139,46,среднее,1,Не женат / не замужем,4,F,сотрудник,0,119516.943993,сделка с автомобилем
4957,1,382970.199936,59,среднее,1,женат / замужем,0,F,пенсионер,0,51079.297866,операции со своей недвижимостью
15569,0,-727.953722,35,среднее,1,женат / замужем,0,F,госслужащий,0,175346.443859,строительство собственной недвижимости


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Датафрейм состоит из 12 столбцов и 21525 строк, в 2 столбцах имеются пропуски значений. Тип данных в таблице int64, float64 и object**

In [2]:
print(f"Доля пропущенных значений в столбце 'total_income' составляет {debts['total_income'].isna().sum() / debts['total_income'].count():.1%}")
print(f"Доля пропущенных значений в столбце 'days_employed' составляет {debts['days_employed'].isna().sum() / debts['days_employed'].count():.1%}")

Доля пропущенных значений в столбце 'total_income' составляет 11.2%
Доля пропущенных значений в столбце 'days_employed' составляет 11.2%


### Шаг 2.1 Заполнение пропусков

In [3]:
debts['total_income'] = debts['total_income'].fillna(debts['total_income'].median())
debts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Пропущенные значения типа NaN**

**В строках одновременно пропущены значения и в total income и в days employed. Количество пропусков одинаково**

**Доля пропусков составляет 11,2 %**

**Возможные причины появления: отсутствие данных о работе либо некорректное внесение данных**

**Медианным значчением заполнять предпочтительнее, так как медианное значение это хначение среднее в числовом ряду, то есть отражает средняя цифру дохода, где половина пользователей выше по доходу, а вторая ниже. В отличии от среднего арифметического дает более точную картину, которая может искажаться артефактами в виде очень большого или очень маленького дохода некоторых пользователей** 

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [4]:
#debts.loc[debts['days_employed'] < 0].info()
debts['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [5]:
display(debts.loc[debts['days_employed'] < 0].sample(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20578,1,-1960.654109,35,ВЫСШЕЕ,0,женат / замужем,0,M,компаньон,0,111055.709283,высшее образование
14601,0,-152.451507,30,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145378.701983,покупка недвижимости
113,0,-6083.994730,52,среднее,1,женат / замужем,0,F,госслужащий,0,120655.098258,образование
15872,0,-600.418900,39,среднее,1,гражданский брак,1,F,госслужащий,1,113489.725351,автомобили
5738,0,-3982.517631,30,среднее,1,в разводе,3,F,компаньон,0,228020.932146,покупка своего жилья
16404,0,-234.024819,50,среднее,1,женат / замужем,0,F,госслужащий,1,137010.946239,жилье
18378,0,-1603.942553,28,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,160470.715974,на покупку подержанного автомобиля
1648,1,-1193.764427,27,среднее,1,женат / замужем,0,F,компаньон,0,224175.433275,сделка с подержанным автомобилем
10783,2,-88.813603,31,среднее,1,женат / замужем,0,M,сотрудник,0,229730.875703,автомобиль
5240,0,-1225.819098,29,среднее,1,Не женат / не замужем,4,M,сотрудник,0,260882.310357,покупка жилья для сдачи


In [6]:
debts.loc[debts['days_employed'] < 0, 'days_employed'] = -1 * debts.loc[debts['days_employed'] < 0, 'days_employed']
display(debts.loc[debts['days_employed'] < 0].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [7]:
debts['days_employed'] = debts['days_employed'].fillna(debts['days_employed'].median())
debts['days_employed'].describe()

count     21525.000000
mean      60378.032733
std      133257.558514
min          24.141633
25%        1025.608174
50%        2194.220567
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

**В days_employed более 15 тысяч отлицательных значений, около 75%, предполагаю, что данные изначально некорректны.**

### Шаг 2.3. Изменение типов данных.

In [8]:
debts['total_income'] = debts['total_income'].astype('int')
debts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [9]:
debts.education.unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [10]:
debts.family_status.unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [11]:
debts.income_type.unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [12]:
debts.purpose.unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [13]:
debts.gender.unique()

array(['F', 'M', 'XNA'], dtype=object)

In [14]:
display(debts[debts.gender == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [15]:
debts.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [16]:
debts.dob_years.value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [17]:
debts.children = debts.children.replace(20, 2)
debts.children = debts.children.replace(-1, 1)

In [18]:
debts.education = debts.education.str.lower()
debts.duplicated().sum()

71

In [19]:
debts.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Проверены остальные столбцы на предмет неявных дубликатов и аномалий. в столбцах с текстовыми значениями использовал unique(), в числовых 
Дубликаты были обнаружены только в столбце education.
в столбце children выявлены аномальные значения -1 и 20. заменил на наиболее близкие по смыслу 1 и 2 соответственно. (их менее 0,05% и они не повлияют на качество исследования но очень смущают меня) 
В dob_years присутствуют люди с возрастом 0, их количество составляет 100 и их можно не принимать во внимание при исследовании.
Можно заменить значением mean для каждой категории занятости.
Количество строк с дубликатами в датафрейме составило 71**

Кажется, что оптимальнее по времени было бы сделать проверку value_counts в цикле для всего датафрейма но не уверен что это удобнее просматривать

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [20]:
#Создал словари, проверил, все работает
edu = debts[['education_id', 'education']]
fam = debts[['family_status_id', 'family_status']]
edu = edu.drop_duplicates().reset_index(drop=True)
fam = fam.drop_duplicates().reset_index(drop=True)
print(edu)
print()
print(fam)

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


In [21]:
debts.drop(['education', 'family_status'], axis=1, inplace=True) 


### Шаг 2.6. Категоризация дохода.

In [22]:
def income_category(row):

    income = row['total_income']
    
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'

In [23]:
debts['total_income_category'] = debts.apply(income_category, axis=1)

In [24]:
debts.total_income_category.value_counts()

C    16087
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [25]:
def purpose_category(row):
    
    purpose = row['purpose']
    
    if 'автом' in purpose:
        return 'операции с автомобилем'
    elif 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'

In [26]:
debts['purpose_category'] = debts.apply(purpose_category, axis=1)
debts.purpose_category.value_counts()

операции с недвижимостью    10840
операции с автомобилем       4315
получение образования        4022
проведение свадьбы           2348
Name: purpose_category, dtype: int64

In [27]:
def pivot_debt(data, index):
    pivot_sorted = data.pivot_table(index=[index], columns='debt', values ='dob_years', aggfunc='count')
    pivot_sorted['ratio'] = pivot_sorted[1] / (pivot_sorted[1] + pivot_sorted[0]) * 100
    return pivot_sorted

In [28]:
pivot_debt(debts, 'children')

debt,0,1,ratio
children,,,
0,13086.0,1063.0,7.512898
1,4420.0,445.0,9.146968
2,1929.0,202.0,9.479118
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


In [29]:
pivot_debt(debts, 'family_status_id')

debt,0,1,ratio
family_status_id,,,
0,11449,931,7.520194
1,3789,388,9.288963
2,897,63,6.562500
3,1110,85,7.112971
4,2539,274,9.740491


In [30]:
pivot_debt(debts, 'total_income_category')

debt,0,1,ratio
total_income_category,,,
A,23,2,8.000000
B,4685,356,7.062091
C,14727,1360,8.454031
D,329,21,6.000000
E,20,2,9.090909


### Ответы на вопросы.

 Есть ли зависимость между количеством детей и возвратом кредита в срок?
**Семьи с 1, 2, 4 детьми возвращают кредит с меньшей вероятностью.**

 Есть ли зависимость между семейным положением и возвратом кредита в срок?
**Да, люди с семейными статусами женат, разведен и вдовец возвращают кредит чаще, чем неженатые и живущие гражданским браком(сожители). Можно предположить, что люди, которые были готовы зарегистрировать свои отношения, более ответственны и планируют свою жизнь тщательнее**

 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
**Да, с увеличением дохода меньшается процент просроченных кредитов. В сегменте дохода C наиболшее количество выданных кредитов, предполагаю что с увеличенным обьемом выборки связан и увеливенных процент просрочек.**
 Как разные цели кредита влияют на его возврат в срок?
**Кредиты на недвижимость возвращают в срок с наибольшей вероятностью. Предположу, что это связано с долгосрочным планированием большей части кредитополучателей**


## Общий вывод:

**При работе с данными были обработаны пропущенные значения, исправлены аномальные значения, удалены неявные дубликаты. Данные были сгруппированы по заданным критериям.
При помощи сводных таблиц оценили критерии которые могут повлиять на своевременный возврат кредита.**

**Рекомендация заказчику, сосредоточиться на привлечении молодых семей без детей и среднего класса**